# PREPROCESSING

In [1]:
# processes a textfile according with skip-gram algorithm
# returns a list in the following format [[7,4],[42,44], ...]

from preprocessing import Preprocess
preprocess = Preprocess()

# TRAINING

In [ ]:
# best trainable after extending the data_rate_limit
# use terminal cmd w/ MAC: "jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000"

import torch
from torch.utils.data import Dataset, DataLoader
from tensorboardX import SummaryWriter
from tqdm import tqdm

from dataset import LoadedDataSet
from model import Word2Vec
from utils import AverageMeter

dimensionSize = 200
num_epochs = 2
lr = 0.001
batchSize = 80

dataset = LoadedDataSet(preprocess.idx_pairs)
loader = DataLoader(dataset, batchSize, shuffle=True) 
model = Word2Vec(preprocess.vocabulary_size, dimensionSize)
optimizer = torch.optim.Adam(model.parameters(), lr)
writer = SummaryWriter()
losses = AverageMeter()

# print('total amount of batches {}'.format(len(idx_pairs) / batchSize))
for e in range(num_epochs):
    losses.reset()
   
    for i, (data, target) in enumerate(tqdm(loader)):   
        n_iter = e * len(dataset) + i

        loss = model.forward(data, target)  
        
#         compute gradient and do optimizer step        
        optimizer.zero_grad()
        loss.backward()
#        think about adding clipping here
#         nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()     
    
        losses.update(loss.item(), batchSize)
        
        if i % 100 == 0:            
            writer.add_scalar('loss', losses.val, n_iter)
            writer.add_scalar('avg_loss', losses.avg, n_iter)
            
writer.close

# SAVING TRAINED MODEL

In [ ]:
import os

if not os.path.exists("SOFTMAX_trained"):
    os.makedirs("SOFTMAX_trained")
    
torch.save(model.input_embeddings(), os.path.join("SOFTMAX_trained", "attempt#1.w2v"))

# LOADING & EVALUATING TRAINED MODEL

In [2]:
# loads the weights of a saved model and calculates and prints the following metrics: SpearManRank, MSE, MAE 
# look @ the data with TensorBoardX "tensorboard --logdir runs"

import torch
import os
from evaluation import Metrics

loadedModel = torch.load(os.path.join("SOFTMAX_trained", "attempt#1.w2v"))
Metrics(loadedModel, preprocess)

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.

mein Spearman: 0.47803712272140936
sein Spearman: 0.760323624228928
mein MAE ist 0.29269872321215323
sein MAE ist 0.24180360721442884
mein MSE ist 0.1208135650397841
sein MSE ist 0.08009566633266534


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U4 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
